## Generate MetaCyc-dependendt files

In [17]:
import cobra
import os
import pandas as pd
import numpy as np
import urllib
import pythoncyc as pc

## Metabolites

met_output_file = 'metabolites.txt'
m_model_file = './iYO844.json'

m_model = cobra.io.load_json_model(m_model_file)

In [18]:
bsub = pc.select_organism('bsub')

In [19]:
print bsub

<PGDB bsub, currently has 0 PFrames>


In [46]:
CPLXS = pc.PGDB.all_protein_complexes(bsub,filter='all')
print CPLXS

[u'|CPLX8J2-77|', u'|CPLX8J2-91|', u'|CPLX-7603|', u'|CPLX8J2-101|', u'|CPLX8J2-119|', u'|CPLX8J2-14|', u'|CPLX8J2-164|', u'|CPLX8J2-179|', u'|CPLX8J2-194|', u'|CPLX8J2-32|', u'|CPLX8J2-48|', u'|CPLX8J2-64|', u'|CPLX8J2-8|', u'|CPLX8J2-94|', u'|CPLX-8351|', u'|CPLX8J2-104|', u'|CPLX8J2-122|', u'|CPLX8J2-142|', u'|CPLX8J2-145|', u'|CPLX8J2-167|', u'|CPLX8J2-181|', u'|CPLX8J2-20|', u'|CPLX8J2-36|', u'|CPLX8J2-50|', u'|CPLX8J2-67|', u'|CPLX8J2-82|', u'|CPLX8J2-97|', u'|CPLX-8648|', u'|CPLX8J2-107|', u'|CPLX8J2-125|', u'|CPLX8J2-148|', u'|CPLX8J2-17|', u'|CPLX8J2-185|', u'|CPLX8J2-23|', u'|CPLX8J2-39|', u'|CPLX8J2-53|', u'|CPLX8J2-70|', u'|CPLX8J2-85|', u'|CPLX-9083|', u'|CPLX8J2-110|', u'|CPLX8J2-128|', u'|CPLX8J2-157|', u'|CPLX8J2-172|', u'|CPLX8J2-188|', u'|CPLX8J2-26|', u'|CPLX8J2-41|', u'|CPLX8J2-56|', u'|CPLX8J2-73|', u'|CPLX8J2-88|', u'|CPLX-5322|', u'|CPLX-9285|', u'|CPLX8J2-113|', u'|CPLX8J2-131|', u'|CPLX8J2-16|', u'|CPLX8J2-175|', u'|CPLX8J2-190|', u'|CPLX8J2-29|', u'|CPLX8J2-44

In [73]:
filename = 'protein_complexes_test.txt'
file = open(filename,'w')
for cplx in CPLXS:
    
    rxn = pc.PGDB.reactions_of_enzyme(bsub,cplx)
    
    cplx_string = str(cplx)
    cplx_string = cplx_string[1:len(cplx_string)-1]
    string = cplx_string + '\t' + 'default_name' + '\t'
    
    genes_of_cplx = pc.PGDB.genes_of_protein(bsub,cplx)
    
    for gene in genes_of_cplx:
        gene_string = str(gene)
        gene_string = gene_string[1:len(gene_string)-1]
        string = string + gene_string +'(1)' + ' AND '
        
    string = string[0:len(string)-4]
    string = string + '\t' + 'M_protein_recon' + '\n'
    file.write(string)
    
file.close()
    
    


[]
[]
[]
[u'|ATPSYN-RXN|']
[u'|TRANS-RXN8J2-112|']
[u'|5.99.1.3-RXN|']
[u'|ASPARTATEKIN-RXN|']
[u'|RXN-17897|', u'|FLAVONADPREDUCT-RXN|']
[u'|RXN-13198|']
[]
[]
[]
[u'|RXN0-901|', u'|RXN-7682|']
[]
[u'|THIAZOLSYN3-RXN|']
[u'|RXN-11136|']
[u'|RIBOFLAVIN-SYN-RXN|']
[u'|TRANS-RXN8J2-114|']
[]
[]
[u'|PGLYCDEHYDROG-RXN|']
[u'|SULFITE-REDUCT-RXN|']
[]
[]
[]
[u'|TRANS-RXN8J2-95|']
[]
[u'|RXN-11213|', u'|1.2.1.27-RXN|', u'|RXN-2902|']
[]
[u'|PENICILLIN-AMIDASE-RXN|']
[]
[u'|GCVP-RXN|']
[u'|5-OXOPROLINASE-ATP-HYDROLYSING-RXN|']
[u'|ACETOOHBUTSYN-RXN|', u'|ACETOLACTSYN-RXN|']
[]
[]
[]
[]
[u'|RXN-17519|', u'|RXN-19755|', u'|RXN-19756|']
[]
[u'|RXN8J2-1|']
[]
[]
[u'|PYRROLINECARBREDUCT-RXN|']
[]
[u'|R145-RXN|']
[]
[]
[u'|ENOYL-ACP-REDUCT-NADH-RXN|', u'|RXN-9558|', u'|RXN-9661|', u'|RXN-9659|', u'|RXN-9660|', u'|RXN-9657|', u'|RXN-9663|', u'|RXN-9658|', u'|RXN-9662|', u'|1.3.1.9-RXN|']
[u'|2.7.7.39-RXN|']
[u'|RXN1K-87|', u'|RXN-19491|', u'|RXN-12354|', u'|PHENYLPYRUVATE-TAUTOMERASE-RXN|', u'|RXN-72

In [ ]:
genes = pc.PGDB.genes_of_protein(bsub,CPLXS[1])
print genes

In [43]:
pc.PGDB.enzymes_of_gene(bsub,genes[0])

[u'|CPLX8J2-9|']

AttributeError: 'list' object has no attribute 'id'